In [ ]:
import cv2 as cv
import numpy as np
import epfl_data2 as epfl_data
import matplotlib.pyplot as plt
from skimage.feature import hog as HoG
from sklearn.preprocessing import normalize
from sklearn.utils import shuffle

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
def flip_angle(angle):
    if angle == 180:
        return 0
    elif angle == 0:
        return 180
    else:
        return 360-angle


data = epfl_data.Data(1,6)

training_images = np.zeros((1327,11016))
training_images_f = np.zeros((1327,11016))
test_images = np.zeros((972,11016))
training_tgts = []
training_tgts_f = []
test_tgts = []

height = 250
width = 376

for d in range(len(data.samples[0])*2):
    image = cv.imread(data.samples[0][d])
    image = image[28:212, 30:348]
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    f_image = np.fliplr(image)
    if d <= 1326:
        image = HoG(image, orientations=8, pixels_per_cell=(16,16))
        image = normalize(image)
        f_image = HoG(f_image, orientations=8, pixels_per_cell=(16,16))
        f_image = normalize(image)
        training_images[d] = image
        training_tgts.append(np.deg2rad(data.samples[1][d]))
        training_images_f[d] = f_image
        training_tgts_f.append(np.deg2rad(flip_angle(data.samples[1][d])))
    else:
        if d-1326 == 972:
            break
        else:
            image = HoG(image, orientations=8, pixels_per_cell=(16, 16))
            image = normalize(image)
            test_images[d-1326] = image
            test_tgts.append(np.deg2rad(data.samples[1][d]))


train = np.append(training_images, training_images_f,axis=0)
for i in range(len(training_tgts)):
    training_tgts.append(training_tgts_f[i])

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv1D

In [ ]:
model = Sequential()
model.add(Dense(1000, input_dim=11016, activation='relu'))
# model.add(Dropout(.5))
model.add(Dense(100, activation='relu'))
# model.add(Dropout(.5))
model.add(Dense(10, activation='relu'))
# model.add(Dropout(.5))
model.add(Dense(10, activation='relu'))
# model.add(Dropout(.5))
model.add(Dense(1))
# model.add(Dropout(.5))

model.compile(optimizer="adam", loss='mean_squared_error', metrics=['accuracy'])

model.fit(train, np.asarray(training_tgts), batch_size=32, epochs=20, verbose=1)

score = model.evaluate(test_images[:-1], np.asarray(test_tgts), verbose=1)
print("\nSCORE:", score)

model.summary()

predictions = model.predict(test_images[:-1])

print("PREDICTIONS:", predictions.T)
print("ACTUAL:", test_tgts)

In [ ]:
print(predictions[10:20].T)
print(test_tgts[10:20])